## Предсказание ридабилити научно-популярных текстов при помощи машинного обучения 

Под ридабилити традицонно понимается "удобочитаемость" текстов, то есть их понятность и легкость чтения. Чаще всего для этого используются различные метрики, основанные на статистических подсчетах средней длины предложений, слов в них, количества "сложных слов" и т.д., и различные соотношения между этими характеристиками.

Оценка ридабилити научно-популярных текстов -- одна из задач в рамках нашего НИСа. Я решила попробовать решить ее при помощи машинного обучения.
Наш корпус составляет около 30 тыс. текстов, выкаченных с различных ресурсов. 

Я взяла **320**  из них (для обучения это, конечно, немного) и разбила на **три класса** по уровню сложности: простые (содержание понятно и можно легко пересказать, не содержит специфических терминов и не требует глубокого знания контекста), средние (рассказывают о какой-то идее или концепции, для понимания нужно знать ряд терминов и/или контекст), сложные (серьезное научное изложение, для понимания необходимо знать контекст, понять о чем речь в полной мере не представляется возможным без спец образования или отдельного изучения предмета).

Я написала [модуль](https://github.com/ana-kuznetsova/Popular-Science-Texts-Compling-research/blob/master/readability/readability_metrics.py) на питоне, который позволяет посчитать следующие метрики:
* *Flesch reading ease (FRE)* -- индекс удобочитаемости Флеша. Считает соотношение общего количество слов, предложений и количества слогов. Чем меньше значение, тем сложнее текст (отсчет идет от 100 и иногда может уходить в минус)

* *Flesh-Kincaid Grade (FKG)* -- считает индекс Флеша и переводит его в шкалу от 0 до 20 (с возможным превышением), ставя в соответствие необходимый уровень образования (5 -- 5 класс, 12 -- выпускник школы, от 14 и выше -- студенты и специалисты, и т.д.). Такой перевод используется и в остальных метриках. 

* *SMOG index* --  считает количество слов, в которых более 3 слогов и их количество в предложениях

* *Coleman-Liau index (CLI )* -- считает не количество слогов, а количество букв в словах и слов в предложениях

* *Dale chall readability score (DCH)* -- количество сложных слов и средняя длина предложений

* *Gunning fog* -- считает сложные слова (более 3 слогов) и их количество в тексте

В работе с этими метриками пришлось учитывать две особенности: во-первых, они разработаны для английского языка, поэтому для русского пришлось подбирать адаптации (считать "сложными" слова, в которых более 4 слогов, а не 3, изучить альтернативные коэффиценты для каждой формулы), а во-вторых, научно-популярные тексты не ориентированы на школьников, а чаще всего предназначены для образованных людей, поэтому такие статистические метрики почти всегда будут присваивать им высокий уровень сложности.

Несмотря на эти ограничения, я постаралась попробовать предсказать уровень сложности текстов на основе этих метрик.

На вход подается набор текстов, у каждого из которого есть метка класса -- 1, 2, или 3.

**Признаками** для машинного обучения стали векторы, полученные благодаря подсчетам описанных выше метрик для каждого текста.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
texts = pd.read_csv('scipop5.csv', encoding = 'utf-8') 
texts = texts.drop(['Unnamed: 0'], axis=1)
train, test = train_test_split(texts, test_size=0.3)

In [5]:
y_train = train['class'] 
y_test = test['class']

In [6]:
train[:10]

,text,class
34,"Агентство НАСА и Роскосмос, обсудив все нюансы...",1
71,У проблемы ГМО много аспектов. Есть медицински...,1
107,Как выглядят миноги. Фрагмен...,1
184,"История нужна, чтобы упорядочить мир. Поток бо...",2
230,В конце января команда охотников з...,2
52,Астрофотографы Гевин Хэффернан и Харун Мехмеди...,1
168,"Как сделать материал толщиной в атом, благодар...",2
38,«Сколько-сколько рукопожатий?» Все ли мы знаем...,1
272,Группа исследователей из трех различных лабора...,3
211,Мы публикуем полную стенограмму лекции директо...,2


In [7]:
import ksslib.readability_metrics as kk #импорт модуля с метриками

In [8]:
def statist_vectors(text):
    FRE = kk.flesch_RE(text)
    FKG = kk.flesch_kincaid_grade(text)
    SMOG = kk.smog_index(text)
    CLI = kk.coleman_liau_index(text)
    DCH = kk.dale_chall_score(text)
    GF = kk.gunning_fog(text)
    return [FRE, FKG, SMOG, CLI, DCH, GF]

In [9]:
def count_me_all(texts):
    return [statist_vectors(text) for text in texts]

In [10]:
x_train = count_me_all(train['text']) #считаем признаки для тестовой и обучающей выборки
x_test = count_me_all(test['text'])

In [11]:
x_train[:10]

[[39.43, 12.6, 13.0, 19.25, 7.65, 15.9],
 [46.01, 11.5, 12.9, 17.97, 7.63, 15.8],
 [12.02, 17.7, 15.4, 19.26, 8.57, 19.3],
 [23.53, 17.5, 16.9, 21.12, 8.93, 21.2],
 [24.75, 15.5, 14.8, 18.27, 8.34, 18.4],
 [40.24, 13.9, 14.8, 18.16, 8.15, 18.3],
 [13.09, 19.9, 18.3, 23.04, 9.46, 23.3],
 [47.52, 12.4, 12.8, 16.59, 7.24, 15.6],
 [12.39, 21.6, 18.9, 20.84, 9.38, 24.4],
 [44.66, 12.6, 13.3, 16.7, 7.55, 16.3]]

In [13]:
from sklearn.preprocessing import StandardScaler #нормируем наши вектора

In [14]:
scaler = StandardScaler()
x_fit = scaler.fit(x_train)
x_transform = scaler.transform(x_train)
x_fit_test = scaler.fit(x_test)
x_transform_test = scaler.transform(x_test)

In [15]:
x_transform [:10]

array([[ 0.78807593, -0.91055367, -0.99793199, -0.3778168 , -0.90105464,
        -1.00313927],
       [ 1.20042634, -1.20775507, -1.04014117, -0.74648468, -0.92195661,
        -1.03200941],
       [-0.929633  ,  0.46738006,  0.01508835, -0.37493658,  0.06043602,
        -0.02155463],
       [-0.20833311,  0.41334344,  0.64822606,  0.16078392,  0.43667149,
         0.52697797],
       [-0.13187909, -0.12702273, -0.23816674, -0.66007814, -0.17993665,
        -0.28138586],
       [ 0.83883639, -0.55931566, -0.23816674, -0.69176054, -0.37850537,
        -0.31025599],
       [-0.86257905,  1.06178285,  1.2391546 ,  0.71378573,  0.99057372,
         1.13325084],
       [ 1.29505387, -0.96459029, -1.08235035, -1.14395473, -1.32954504,
        -1.08974968],
       [-0.90644612,  1.52109409,  1.49240968,  0.08013782,  0.90696583,
         1.45082234],
       [ 1.11582558, -0.91055367, -0.87130445, -1.11227233, -1.0055645 ,
        -0.88765873]])

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report

Для решения данной задачи я решила опробовать мектор опорных векторов и метод k-ближайших соседей. Смотрим, что получается

In [17]:
model_svc = SVC()
model_svc.fit(x_transform, y_train)

expected_svc = y_test
predicted_svc = model_svc.predict(x_transform_test)

print(metrics.classification_report(expected_svc, predicted_svc))
print(metrics.confusion_matrix(expected_svc, predicted_svc))

             precision    recall  f1-score   support

          1       0.62      0.60      0.61        40
          2       0.42      0.63      0.51        35
          3       0.20      0.05      0.08        21

avg / total       0.45      0.49      0.45        96

[[24 15  1]
 [10 22  3]
 [ 5 15  1]]


In [18]:
model_kn = KNeighborsClassifier()
model_kn.fit(x_transform, y_train)

expected = y_test
predicted = model_kn.predict(x_transform_test)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       0.49      0.68      0.57        40
          2       0.47      0.49      0.48        35
          3       0.20      0.05      0.08        21

avg / total       0.42      0.47      0.43        96

[[27  9  4]
 [18 17  0]
 [10 10  1]]


Результаты выглядят не очень впечатляющими. Метод опорных векторов в целом справилс чуть лучше, примечательно, что с чуть большей точностью, чем все остальное, определяются тексты первого класса, и очень плохо -- третьего.

В целом было бы ошибочно надеяться, что статистические метрики определяют сложность текстапо такому же принципу, что и человек. Возможным решением было бы создание дополнительных признаков: подсчет количества терминов, абстрактных и разговорных выражений, а также тональность текста (здесь можно предположить, что чем эмоциональнее что-либо объясняется, тем более доступным оно кажется читателю). 

В целом данное решение (точнее, зайдествованный для него модуль) полезно для оценки именно статистических характеристик для русского языка, т.к. в него вошли самые актуальные коэффициенты и наработки по этой теме. В то же время этот результат показывает, что машинное обучение только на количественных признаках неэффективно и статистические параметры не отражают человеческого представления о сложности текста, и, по всей видимости, для более точного результата модель обучения необходимо усложнить.